In [ ]:
import sys
import csv
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)

from argparse import ArgumentParser
import itertools as it

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline
from pipeline.utils import validate

In [ ]:
df_dict = {}

for prot in ["tyk2","mcl1","p38"]:
    file = f"/home/anna/Documents/benchmark/extracted/{prot}/perturbing_overlap.dat"
    df = pd.read_csv(file)
    df_dict[prot] = df

frames = []
for key in df_dict.keys():
    frames.append(df_dict[key])

all_df = pd.concat(frames)
df = all_df
# df = df_dict["p38"]

In [ ]:
# histogram of failed run in terms of perturbing atoms

df_has = df[df["percen_overlap_okay"] >= 0]
df_none = pd.merge(df_has, df, how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
print(len(df_none))
df_none["perturbing_atoms"].plot.hist(bins=10)
plt.xlabel("perturbing atoms")

In [ ]:
df_plot = df.dropna()

df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                     c='too_small_avg',
                     colormap='viridis')
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("percentage of okay overlap")

In [ ]:

df_plot.plot.scatter("perturbing_atoms","too_small_avg",
                     c='percen_overlap_okay',
                     colormap='viridis')
# plt.title(f"")
# plt.xlabel("no. of perturbing atoms")
# plt.ylabel("average no. of too small off-diagonals per leg")

In [ ]:
df_plot.plot.scatter("percen_overlap_okay","too_small_avg",
                     c='perturbing_atoms',
                     colormap='viridis')

In [ ]:
# exclude outliers as needed
df3 = df_plot[df_plot["diff_to_exp"] >= 20]
df_out = pd.merge(df3, df_plot, how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

# df_out = df_plot

df_out.plot.scatter("perturbing_atoms","diff_to_exp",
                     c='percen_overlap_okay',
                     colormap='viridis')

In [ ]:
columns = ["perturbing_atoms","percen_overlap_okay","too_small_avg"]
bins = [6, 3, 5]
for column,bin in zip(columns, bins):
    fig = plt.figure()
    df_plot = df[column]
    df_plot.plot.hist(subplots=True, bins=bin)
    plt.title(column)


In [ ]:
# for per engine
eng = "GROMACS"
df2 = df[df["engine"] == eng]
df_plot = df2.dropna()
df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                     c='too_small_avg',
                     colormap='viridis')
df_plot.plot.scatter("perturbing_atoms","too_small_avg",
                     c='percen_overlap_okay',
                     colormap='viridis')
df_plot.plot.scatter("percen_overlap_okay","too_small_avg",
                     c='perturbing_atoms',
                     colormap='viridis')

In [ ]:
engs = ["AMBER","SOMD","GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines._set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    eng_dict[eng] = df2

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                        c=col_dict[eng])

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                    c=col_dict["AMBER"])
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                    c=col_dict["SOMD"], ax=ax1)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter("perturbing_atoms","percen_overlap_okay",
                    c=col_dict["GROMACS"], ax=ax1)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)

In [ ]:
engs = ["AMBER","SOMD","GROMACS"]
eng_dict = {}

col_dict = pipeline.analysis.plotting_engines._set_colours()

for eng in engs:
    df2 = df[df["engine"] == eng]
    df3 = df2[df2["diff_to_exp"] >= 5]
    df4 = pd.merge(df3, df2, how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)
    eng_dict[eng] = df4

for eng in engs:
    df_plot = eng_dict[eng].dropna()
    ax = df_plot.plot.scatter("perturbing_atoms","diff_to_exp",
                        c=col_dict[eng])

df_plot = eng_dict["AMBER"].dropna()
ax1 = df_plot.plot.scatter("perturbing_atoms","diff_to_exp",
                    c=col_dict["AMBER"])
df_plot = eng_dict["SOMD"].dropna()
ax2 = df_plot.plot.scatter("perturbing_atoms","diff_to_exp",
                    c=col_dict["SOMD"], ax=ax1)
df_plot = eng_dict["GROMACS"].dropna()
ax3 = df_plot.plot.scatter("perturbing_atoms","diff_to_exp",
                    c=col_dict["GROMACS"], ax=ax1)
plt.legend(col_dict, loc="upper right")
print(ax1 == ax2 == ax3)